In [ ]:
import import_ipynb
import numpy as np
from scipy.constants import epsilon_0, mu_0

# Project 2: Coaxial Longitudinal Problem 

Considere um cabo coaxial com dois dielétricos apresentado na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/coaxial_longitudinal.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of Coaxial Problem.</figcaption>
</figure>

# `uniform_current_density()`

In [ ]:
def uniform_current_density(e, mesh_data, Icc):
    """
    Calcula a densidade de corrente uniforme nas regiões condutoras
    de um cabo coaxial.

    Parâmetros:
    - I: Corrente total (A)
    - a: Raio do condutor interno (m)
    - b: Raio interno da blindagem (m)
    - c: Raio externo da blindagem (m)

    Retorna:
    - J_inner: Densidade de corrente no núcleo (A/m²)
    - J_outer: Densidade de corrente na blindagem (A/m²)
    """
    material_tag = mesh_data['conn_data'][e]['material']['tag']
    radii = {'a': 1e-2, 'b': np.sqrt(3) * 1e-2, 'c': 2e-2}  

    # Área do condutor interno
    A_inner = np.pi * radii['a']**2
    
    # Área da blindagem
    A_outer = np.pi * (radii['c']**2 - radii['b']**2)
    
    # Densidades de corrente
    if material_tag == 201:
        Jz = Icc / A_inner
    elif material_tag == 202:
        Jz = 0
    elif material_tag == 203:
        Jz = -Icc / A_outer

    return Jz

# `calculate_current_density()`

In [ ]:
def calculate_current_density(e, mesh_data, element_type, Icc):
    """
    Calcula a densidade de corrente uniforme por elemento.
    
    Parâmetros:
    - conn_data: Lista de elementos, cada um com 'conn_list' e 'material'.
    - mesh_nodes: Lista de nós com 'TagID' e 'global_coord'.
    - total_current: Corrente total aplicada (A).
    
    Retorna:
    - Lista de densidades de corrente por elemento.
    """
    # Desempacotar o tipo de elemento
    type_name, order = element_type
    
    # Lista de coordenadas globais dos nós
    nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
    conn = [element['conn_list'] for element in mesh_data['conn_data']]
    
    # Nodes coordinates of element e
    ai = [nodes[node - 1] for node in conn[e]]; 

    if type_name == 'Triangle':
        # Para elementos P1, P2 ou P3, usar os 3 vértices principais
        if order in [1, 2, 3]:
            ai = ai[:3]
        else:
            raise ValueError("Tipo de elemento não suportado para triângulos: P1, P2 ou P3 são esperados.")

        # Obter as coordenadas dos vértices do triângulo
        (x1, y1), (x2, y2), (x3, y3) = ai
    
        # Calcular a área do triângulo
        e_area = 0.5 * abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
        
    # Corrente do elemento (assumindo corrente uniforme pela área total)
    material_tag = mesh_data['conn_data'][e]['material']['tag']
    if material_tag == 201:  # Núcleo
        Jz = Icc / e_area
    elif material_tag == 202:  # Dielétrico
        Jz = 0
    elif material_tag == 203:  # Blindagem
        Jz = -Icc / e_area             
    
    return Jz

# `apply_physics()`

In [ ]:
def apply_physics(mesh_data, element_type):
    """
    Adiciona uma nova chave 'source' a cada elemento em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Lista de células e suas conectividades
    type, order = element_type
    
    for e, cell in enumerate(mesh_data['conn_data']):
        # Adicionar a nova chave 'source'
        cell['source'] = {
            'type': 'uniform_current_density_Jz',
            'value': uniform_current_density(e, mesh_data, Icc=1)
        }

        # Adicionar nova chave 'a_value' 
        mur = cell['material']['relative_magnetic_permeability'] 
        cell['stiffness_a_value'] = 1/(mur * mu_0)

    return mesh_data

# `potential_and_gradient()`

In [ ]:
def potential_and_gradient(x, y):
    return 0, np.zeros((2,1))

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.